<a href="https://colab.research.google.com/github/ashikita/pdf_adjustment/blob/main/pdf_encrypt_copy_restrict.py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# -*- coding: utf-8 -*-
"""
PDFの本文コピーを制限する

"""

# ---- 依存パッケージ（必要なら pip で自動インストール）----
import sys
import subprocess

def _pip_install(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", pkg])

try:
    import pypdf  # 推奨ライブラリ
    from pypdf import PdfReader, PdfWriter, Permissions
    _LIB = "pypdf"
except ImportError:
    _pip_install("pypdf")
    import pypdf
    from pypdf import PdfReader, PdfWriter, Permissions
    _LIB = "pypdf"

print(f"[INFO] Using {_LIB} {pypdf.__version__}")

# === 設定ブロック ===
# 目的：本文コピーを制限（閲覧は無制限）、印刷は許可、保護解除はオーナーパスワードでのみ可能
USER_PASSWORD = ""                     # ★閲覧を無制限にするため空文字にする（= 開くときはパスワード不要）
OWNER_PASSWORD = "set_a_strong_password_here"  # ★必須：保護解除（権限変更）時に必要。強固な値にしてください。
ALLOW_ACCESSIBILITY = True             # スクリーンリーダー向けテキスト抽出を許可する場合 True
ALGORITHM_PREFS = ("AES-256", "AES-128")  # 利用可能な暗号化アルゴリズムの優先順（pypdf は AES 系を推奨）

# === ここから実装 ===
import os
import time
import glob

# 印刷のみ許可・コピー禁止（必要に応じてアクセシビリティ抽出は許可）
# ※ Permissions に "EXTRACTION/COPY" を含めない＝コピー禁止
allowed_permissions = {Permissions.PRINTING}
if ALLOW_ACCESSIBILITY and hasattr(Permissions, "ACCESSIBILITY"):
    allowed_permissions.add(Permissions.ACCESSIBILITY)

def encrypt_with_best_algorithm(writer, user_pwd, owner_pwd, perms):
    """
    pypdf で permissions + algorithm を確実に適用する。
    - user_pwd は空文字で OK（閲覧時にパスワード不要、ただし権限は適用される）
    - owner_pwd は必須（保護解除に必要）
    """
    if not owner_pwd:
        raise ValueError("OWNER_PASSWORD は必須です（保護解除用）。空にしないでください。")

    # まず algorithm を指定して試す（新しめの pypdf）
    for algo in ALGORITHM_PREFS:
        try:
            writer.encrypt(
                user_password=user_pwd,       # ← 空文字で閲覧は無制限
                owner_password=owner_pwd,     # ← 非空必須（保護解除用）
                permissions=perms,            # ← コピーを含めないセット（印刷は許可）
                algorithm=algo,
            )
            return
        except TypeError:
            # 古い pypdf だと algorithm 引数が未対応の場合あり → 次の手へ
            pass
        except Exception:
            # アルゴリズム非対応など → 次の候補へ
            pass

    # algorithm を明示しないパス（やや古い pypdf）
    writer.encrypt(
        user_password=user_pwd,
        owner_password=owner_pwd,
        permissions=perms,
    )

def process_pdf(in_path, out_path):
    # 読み込み（暗号化済み PDF への対処：空パスで開けるタイプに一応対応）
    reader = PdfReader(in_path)
    if getattr(reader, "is_encrypted", False):
        try:
            if hasattr(reader, "decrypt"):
                reader.decrypt("")
        except Exception:
            pass

    writer = PdfWriter()

    # ページを全コピー
    for page in reader.pages:
        writer.add_page(page)

    # メタデータ継承（可能な範囲で）
    try:
        meta = getattr(reader, "metadata", None)
        if meta:
            writer.add_metadata({k: v for k, v in meta.items() if v is not None})
    except Exception:
        pass

    # 暗号化＆権限設定（コピー禁止・印刷可）
    encrypt_with_best_algorithm(
        writer,
        USER_PASSWORD,
        OWNER_PASSWORD,
        allowed_permissions,
    )

    # 書き出し
    with open(out_path, "wb") as f:
        writer.write(f)

def main():
    os.makedirs("output", exist_ok=True)

    # 作業ディレクトリ直下の PDF を対象（output 内は対象外）
    candidates = []
    for pattern in ("*.pdf", "*.PDF", "*.Pdf", "*.pDf", "*.pdF"):
        candidates.extend(glob.glob(pattern))
    # 念のため重複排除＆ソート
    candidates = sorted(set(candidates))

    if not candidates:
        print("INFO: 対象となる PDF が見つかりませんでした。")
        return

    for pdf_path in candidates:
        start = time.perf_counter()
        status = "SUCCESS"
        err_msg = ""
        try:
            out_path = os.path.join("output", os.path.basename(pdf_path))
            process_pdf(pdf_path, out_path)
        except Exception as e:
            status = "ERROR"
            err_msg = str(e)
        elapsed = time.perf_counter() - start

        # 要求仕様：ファイル名、成功・エラーのステータス、かかった時間を標準出力
        if status == "SUCCESS":
            print(f"{os.path.basename(pdf_path)},SUCCESS,{elapsed:.3f}s")
        else:
            print(f"{os.path.basename(pdf_path)},ERROR,{elapsed:.3f}s,{err_msg}")

if __name__ == "__main__":
    main()


In [ ]:
# outputディレクトリ内のファイルを削除

!rm -f output/*